# IF3070 Foundations of Artificial Intelligence | Tugas Besar 2

This notebook serves as a template for the assignment. Please create a copy of this notebook to complete your work. You can add more code blocks, markdown blocks, or new sections if needed.


Group Number: xx

Group Members:
- Name (NIM)
- Name (NIM)
- ...

## Import Libraries

In [73]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Import other libraries if needed
# pd.set_option('display.max_colwidth', None) 
# pd.set_option('display.max_rows', None)     
# pd.set_option('display.max_columns', None)  

pd.reset_option('display.max_colwidth')
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

## Import Dataset

In [91]:
# Example of reading a csv file from a gdrive link

# Take the file id from the gdrive file url
# https://drive.google.com/file/d/1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU/view?usp=sharing => The file id is 1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU
# and then put it in this format:
# https://drive.google.com/uc?id={file_id}
# Don't forget to change the access to public

# df = pd.read_csv('https://drive.google.com/uc?id=1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU')
# df.head()

sample = pd.read_csv('https://drive.google.com/uc?id=14PntjRvR-yo95ZSHj9B5yjrhz8EDWm12')
test = pd.read_csv('https://drive.google.com/uc?id=1KHe0M57H7pjt_SLqTPQAa18Bxr_HbjnT')
# train = pd.read_csv('https://drive.google.com/uc?id=1thHVhP-KClx4K4JxO_ctokR7P14DxOOB')

train = pd.read_csv('https://drive.google.com/uc?id=1q8FJa_HN74Nyjme_tpUFJ1ncQiPg1ffI')

# sample.head()
# test.head()
train.head()

,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,521848.txt,https://www.southbankmosaics.com,31,www.southbankmosaics.com,24,0,com,100.0,1.000000,0.522907,...,0,0,1,34,20,28,119,0,124,1
1,31372.txt,https://www.uni-mainz.de,23,www.uni-mainz.de,16,0,de,100.0,0.666667,0.032650,...,0,0,1,50,9,8,39,0,217,1
2,597387.txt,https://www.voicefmradio.co.uk,29,www.voicefmradio.co.uk,22,0,uk,100.0,0.866667,0.028555,...,0,0,1,10,2,7,42,2,5,1
3,554095.txt,https://www.sfnmjournal.com,26,www.sfnmjournal.com,19,0,com,100.0,1.000000,0.522907,...,1,1,1,3,27,15,22,1,31,1
4,151578.txt,https://www.rewildingargentina.org,33,www.rewildingargentina.org,26,0,org,100.0,1.000000,0.079963,...,1,0,1,244,15,34,72,1,85,1


# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [88]:
# Split training set and validation set here, store into variables train_set and val_set.
# Remember to also keep the original training set before splitting. This will come important later.
# train_set, val_set = ...

# Split the data
train_set, val_set = train_test_split(
    train,
    test_size=0.2,  # 20% for validation
    stratify=train['label'],  # Ensure the same proportion of '0' and '1'
    random_state=42  # Ensures reproducibility
)

In [89]:
# Display the shapes to verify
print("Original dataset shape:", train.shape)
print("Training set shape:", train_set.shape)
print("Validation set shape:", val_set.shape)

# Verify the distribution of the target variable
print("Label distribution in training set:\n", train_set['label'].value_counts(normalize=True))
print("Label distribution in validation set:\n", val_set['label'].value_counts(normalize=True))

Original dataset shape: (140404, 56)
Training set shape: (112323, 56)
Validation set shape: (28081, 56)
Label distribution in training set:
 1    0.924833
0    0.075167
Name: label, dtype: float64
Label distribution in validation set:
 1    0.924825
0    0.075175
Name: label, dtype: float64


# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [84]:
# TLDLength

# NoOfSubDomain

# HasObfuscation

# NoOfObfuscatedChar

# NoOfLettersInURL

# NoOfDegitsInURL

# NoOfEqualsInURL

# NoOfQMarkInURL

# NoOfAmpersandInURL

# NoOfOtherSpecialCharsInURL

# HasFavicon

# Robots

# IsResponsive

# NoOfURLRedirect

# NoOfSelfRedirect

# HasDescription

# NoOfPopup

# NoOfiFrame

# HasExternalFormSubmit

# HasSocialNet

# HasSubmitButton

# HasHiddenFields

# HasPasswordField

# Bank

# Pay

# Crypto

# HasCopyrightInfo

# NoOfImage

# NoOfCSS

# NoOfJS

# NoOfSelfRef

# NoOfEmptyRef

# NoOfExternalRef

In [85]:
# Approach for TLDLength:
# Directly calculate TLDLength from the length of the TLD column.
def fill_tld_length(df):
    df['TLDLength'] = df['TLD'].str.len()
    return df

# Approach for NoOfObfuscatedChar:
# Parse obfuscated characters (e.g., %xx, \x, \u) directly from the URL.
# If HasObfuscation or ObfuscationRatio is missing, calculate obfuscation from the URL.
def get_obfuscated_chars(url):
    obfuscated_chars = []
    common_patterns = [r'%[0-9a-fA-F]{2}', r'\\x[0-9a-fA-F]{2}', r'\\u[0-9a-fA-F]{4}']
    for pattern in common_patterns:
        obfuscated_chars.extend(re.findall(pattern, url))
    return len(obfuscated_chars)

def fill_no_of_obfuscated_char(df):
    df['NoOfObfuscatedChar'] = df.apply(
        lambda row: get_obfuscated_chars(row['URL'])
        if pd.isnull(row['HasObfuscation']) or pd.isnull(row['ObfuscationRatio'])
        else round(row['ObfuscationRatio'] * row['URLLength']), axis=1
    )
    return df

# Approach for HasObfuscation:
# Set HasObfuscation to 1 if obfuscated characters exist in the URL.
def fill_has_obfuscation(df):
    df['HasObfuscation'] = df['NoOfObfuscatedChar'].apply(lambda x: 1 if x > 0 else 0)
    return df

# Approach for NoOfSubDomain:
# Count subdomains in the Domain column by counting dots (.) and subtracting 1.
def fill_no_of_subdomain(df):
    df['NoOfSubDomain'] = df['NoOfSubDomain'].where(
        ~df['NoOfSubDomain'].isnull(),  
        df['Domain'].str.count(r'\.') - 1 
    )
    return df

# Approach for NoOfLettersInURL:
# Parse the number of letters (alphabetic characters) in the URL directly.
def fill_no_of_letters_in_url(df):
    # Only compute for missing values
    def count_letters_excluding_prefix(url):
        # Remove http, https, and www
        url_without_prefix = re.sub(r'^(https?://)?(www\.)?', '', url)
        # Count alphabetic characters using re.findall and subtract 1
        return max(len(re.findall(r'[a-zA-Z]', url_without_prefix)), 0)

    df['NoOfLettersInURL'] = df['NoOfLettersInURL'].where(
        ~df['NoOfLettersInURL'].isnull(),  # Keep non-missing values
        df['URL'].apply(count_letters_excluding_prefix)  # Apply custom function
    )
    return df

# Approach for NoOfDegitsInURL:
# Parse the number of digits in the URL directly.
def fill_no_of_digits_in_url(df):
    # Define a helper function to count digits excluding prefix
    def count_digits_excluding_prefix(url):
        # Remove http, https, and www
        url_without_prefix = re.sub(r'^(https?://)?(www\.)?', '', url)
        # Count digits in the remaining part of the URL
        return len(re.findall(r'[0-9]', url_without_prefix))
    
    # Apply the helper function to calculate the number of digits
    df['NoOfDegitsInURL'] = df['NoOfDegitsInURL'].where(
        ~df['NoOfDegitsInURL'].isnull(),  # Keep non-missing values
        df['URL'].apply(count_digits_excluding_prefix)  # Calculate only for missing values
    )
    return df

# Approach for NoOfEqualsInURL:
# Count occurrences of = in the URL.
def fill_no_of_equals_in_url(df):
    df['NoOfEqualsInURL'] = df['URL'].str.count('=')
    return df

# Approach for NoOfQMarkInURL:
# Count occurrences of ? in the URL.
def fill_no_of_qmark_in_url(df):
    # Define a helper function to count `?` excluding any escape misinterpretation
    def count_qmarks(url):
        # Remove http, https, and www
        url_without_prefix = re.sub(r'^(https?://)?(www\.)?', '', url)
        # Count occurrences of `?`
        return url_without_prefix.count('?')
    
    # Apply the helper function to calculate the number of `?`
    df['NoOfQMarkInURL'] = df['NoOfQMarkInURL'].where(
        ~df['NoOfQMarkInURL'].isnull(),  # Keep non-missing values
        df['URL'].apply(count_qmarks)  # Calculate only for missing values
    )
    return df

# Approach for NoOfAmpersandInURL:
# Count occurrences of & in the URL.
def fill_no_of_ampersand_in_url(df):
    df['NoOfAmpersandInURL'] = df['URL'].str.count(r'&')
    return df

# Approach for NoOfOtherSpecialCharsInURL:
# Count occurrences of special characters excluding a-zA-Z0-9, &, ?, and =.
def fill_no_of_other_special_chars_in_url(df):
    # Define a helper function to count special characters excluding a-zA-Z0-9, &, ?, and =
    def count_other_special_chars(url):
        # Remove http, https, and www
        url_without_prefix = re.sub(r'^(https?://)?(www\.)?', '', url)
        # Use regex to count special characters excluding a-zA-Z0-9, &, ?, and =
        return len(re.findall(r'[^a-zA-Z0-9&?=]', url_without_prefix))
    
    # Only compute for missing values in the column
    df['NoOfOtherSpecialCharsInURL'] = df['NoOfOtherSpecialCharsInURL'].where(
        ~df['NoOfOtherSpecialCharsInURL'].isnull(),  # Keep non-missing values
        df['URL'].apply(count_other_special_chars)  # Calculate only for missing values
    )
    return df

# Function to fill all features by parsing data from the URL and related columns.
def fill_features_from_url(df):
    df = fill_tld_length(df)
    df = fill_no_of_obfuscated_char(df)
    df = fill_has_obfuscation(df)
    df = fill_no_of_subdomain(df)
    df = fill_no_of_letters_in_url(df)
    df = fill_no_of_digits_in_url(df)
    df = fill_no_of_equals_in_url(df)
    df = fill_no_of_qmark_in_url(df)
    df = fill_no_of_ampersand_in_url(df)
    df = fill_no_of_other_special_chars_in_url(df)
    return df

In [97]:
def fill_missing_values(df):
    # Helper function to safely compute mode
    def safe_mode(series):
        if series.dropna().empty:  # Check if the column has no non-NaN values
            return 0  # Default value for columns with all NaN (adjust as needed)
        return series.mode()[0]

    # Helper function to predict values based on correlated variables
    def predict_based_on_correlation(row, correlated_features, threshold=0.5):
        total_score = 0
        count = 0
        for feature in correlated_features:
            if pd.notnull(row[feature]):
                total_score += row[feature]
                count += 1
        if count == 0:  # If no correlated features have values, fallback to threshold
            return safe_mode(pd.Series(correlated_features))
        return 1 if (total_score / count) >= threshold else 0

    # Fill missing values for each column

    # < 0.3 correlation with label
    df['NoOfURLRedirect'] = df['NoOfURLRedirect'].fillna(df['NoOfURLRedirect'].median())
    df['NoOfSelfRedirect'] = df['NoOfSelfRedirect'].fillna(0)
    df['NoOfPopup'] = df['NoOfPopup'].fillna(0)
    df['NoOfiFrame'] = df['NoOfiFrame'].fillna(0)
    df['HasExternalFormSubmit'] = df['HasExternalFormSubmit'].fillna(safe_mode(df['HasExternalFormSubmit']))
    df['HasPasswordField'] = df['HasPasswordField'].fillna(safe_mode(df['HasPasswordField']))
    df['Bank'] = df['Bank'].fillna(0)
    df['Crypto'] = df['Crypto'].fillna(0)
    df['NoOfImage'] = df['NoOfImage'].fillna(df['NoOfImage'].median())
    df['NoOfCSS'] = df['NoOfCSS'].fillna(df['NoOfCSS'].median())
    df['NoOfEmptyRef'] = df['NoOfEmptyRef'].fillna(0)
    df['NoOfExternalRef'] = df['NoOfExternalRef'].fillna(df['NoOfExternalRef'].median())

    # < 0.5 correlation with label
    df['Robots'] = df['Robots'].fillna(safe_mode(df['Robots']))
    df['Pay'] = df['Pay'].fillna(safe_mode(df['Pay']))
    df['NoOfJS'] = df['NoOfJS'].fillna(df['NoOfJS'].median())
    df['NoOfSelfRef'] = df['NoOfSelfRef'].fillna(0)
    df['HasFavicon'] = df['HasFavicon'].fillna(safe_mode(df['HasFavicon'])) 

    # >= 0.45 correlation with label, use correlated variables for imputation
    df['HasSocialNet'] = df['HasSocialNet'].where(
        ~df['HasSocialNet'].isnull(),
        df.apply(lambda row: predict_based_on_correlation(row, ['URLSimilarityIndex']), axis=1)
    )
    
    df['HasHiddenFields'] = df['HasHiddenFields'].where(
        ~df['HasHiddenFields'].isnull(),
        df.apply(lambda row: predict_based_on_correlation(row, ['HasSocialNet', 'HasSubmitButton']), axis=1)
    )
    
    df['HasSubmitButton'] = df['HasSubmitButton'].where(
        ~df['HasSubmitButton'].isnull(),
        df.apply(lambda row: predict_based_on_correlation(row, ['HasSocialNet', 'HasHiddenFields']), axis=1)
    )
    
    df['HasCopyrightInfo'] = df['HasCopyrightInfo'].where(
        ~df['HasCopyrightInfo'].isnull(),
        df.apply(lambda row: predict_based_on_correlation(row, ['URLSimilarityIndex', 'HasSocialNet', 'HasSubmitButton']), axis=1)
    )
    
    df['HasDescription'] = df['HasDescription'].where(
        ~df['HasDescription'].isnull(),
        df.apply(lambda row: predict_based_on_correlation(row, ['URLSimilarityIndex', 'HasSocialNet', 'HasCopyrightInfo']), axis=1)
    )
    
    df['IsResponsive'] = df['IsResponsive'].where(
        ~df['IsResponsive'].isnull(),
        df.apply(lambda row: predict_based_on_correlation(row, ['IsHTTPS', 'HasSocialNet', 'HasCopyrightInfo', 'HasDescription']), axis=1)
    )
    
    return df


In [94]:
def compare_train_and_copy(original_df, imputed_df, feature_columns):
    """
    Compare original DataFrame with imputed DataFrame and evaluate differences.
    
    Args:
    - original_df (DataFrame): The original DataFrame with missing values.
    - imputed_df (DataFrame): The DataFrame after imputation.
    - feature_columns (list): List of columns to compare.

    Returns:
    - results (DataFrame): DataFrame summarizing the changes per column.
    """
    results = []
    for column in feature_columns:
        original_values = original_df[column]
        imputed_values = imputed_df[column]
        
        # Count differences
        num_missing_original = original_values.isnull().sum()
        num_missing_imputed = imputed_values.isnull().sum()
        num_changed = (original_values != imputed_values).sum()
        
        # Collect results
        results.append({
            "Feature": column,
            "Missing Before": num_missing_original,
            "Missing After": num_missing_imputed,
            "Values Changed": num_changed
        })
    
    return pd.DataFrame(results)


# Create a copy of the train dataset
train_copy = train.copy()

# Initialize the specified columns with NaN values
features_to_test = [
    "TLDLength", "NoOfSubDomain", "HasObfuscation", "NoOfObfuscatedChar", 
    "NoOfLettersInURL", "NoOfDegitsInURL", "NoOfEqualsInURL", "NoOfQMarkInURL",
    "NoOfAmpersandInURL", "NoOfOtherSpecialCharsInURL", "HasFavicon", "Robots", 
    "IsResponsive", "NoOfURLRedirect", "NoOfSelfRedirect", "HasDescription",
    "NoOfPopup", "NoOfiFrame", "HasExternalFormSubmit", "HasSocialNet", 
    "HasSubmitButton", "HasHiddenFields", "HasPasswordField", "Bank", "Pay", 
    "Crypto", "HasCopyrightInfo", "NoOfImage", "NoOfCSS", "NoOfJS", 
    "NoOfSelfRef", "NoOfEmptyRef", "NoOfExternalRef"
]

# Ensure all the specified columns exist and are initialized with NaN
for feature in features_to_test:
    train_copy[feature] = np.nan

# Step 2: Impute missing values using the provided function { Fungsi cleaning nya }
train_copy = fill_features_from_url(train_copy)
train_copy = fill_missing_values(train_copy)

# Step 3: Evaluate imputation
evaluation_results = compare_train_and_copy(train, train_copy, features_to_test)

# Display results
# Display the evaluation results
print(evaluation_results)

C:\Users\FAVIAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\FAVIAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\FAVIAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\FAVIAN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty sli

                       Feature  Missing Before  Missing After  Values Changed
0                    TLDLength               0              0               0
1                NoOfSubDomain               0              0               0
2               HasObfuscation               0              0             402
3           NoOfObfuscatedChar               0              0             797
4             NoOfLettersInURL               0              0          172772
5              NoOfDegitsInURL               0              0            1210
6              NoOfEqualsInURL               0              0             249
7               NoOfQMarkInURL               0              0              12
8           NoOfAmpersandInURL               0              0            3172
9   NoOfOtherSpecialCharsInURL               0              0           16653
10                  HasFavicon               0              0           85303
11                      Robots               0              0   

In [96]:
def compare_specific_columns(original_df, imputed_df, column_names):
    """
    Compare specific columns between original and imputed DataFrames.

    Args:
    - original_df (DataFrame): The original DataFrame.
    - imputed_df (DataFrame): The imputed DataFrame.
    - column_names (list): List of column names to compare.

    Returns:
    - DataFrame with rows where values differ in the specified columns.
    """
    # Filter rows where any specified column's values are different
    differing_rows = original_df[column_names].ne(imputed_df[column_names]).any(axis=1)
    return original_df.loc[differing_rows, column_names], imputed_df.loc[differing_rows, column_names]

# Columns to compare
columns_to_compare = ['URL', 'URLSimilarityIndex', 'HasSocialNet']
# 'IsResponsive', 'HasDescription'

# Compare specific columns and get the differing rows
original_differing, imputed_differing = compare_specific_columns(train, train_copy, columns_to_compare)

# Display the differing rows for inspection
print("Original DataFrame Differing Rows:")
print(original_differing)
print("\nImputed DataFrame Differing Rows:")
print(imputed_differing)


Original DataFrame Differing Rows:
                                                      URL  URLSimilarityIndex  \
0                        https://www.southbankmosaics.com          100.000000   
2                          https://www.voicefmradio.co.uk          100.000000   
11                                http://www.teramill.com           82.644628   
12                           https://www.socialpolicy.org          100.000000   
15                              https://www.brightika.com          100.000000   
...                                                   ...                 ...   
235783        https://bancolombia.com1home0892.repl.co/?2           47.074830   
235784                 https://aol-108318.weeblysite.com/           66.720000   
235785                  https://www.virginiakraljevic.com          100.000000   
235786                     https://www.chilconference.org          100.000000   
235793  https://patient-cell-40f5.updatedlogmylogin.wo...           28.157

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [69]:
# Write your code here

### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [70]:
# Write your code here

### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [71]:
# Write your code here

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [11]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [12]:
# Write your code here

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [13]:
# Write your code here

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [14]:
# Write your code here

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [ ]:
# from sklearn.pipeline import Pipeline

# # Note: You can add or delete preprocessing components from this pipeline

# pipe = Pipeline([("imputer", FeatureImputer()),
#                  ("featurecreator", FeatureCreator()),
#                  ("scaler", FeatureScaler()),
#                  ("encoder", FeatureEncoder())])

# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

In [ ]:
# # Your code should work up until this point
# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

or create your own here

In [ ]:
# Write your code here

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `label`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [ ]:
# Type your code here

## B. Naive Bayes

In [ ]:
# Type your code here

## C. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [ ]:
# Type your code here

## D. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `label` column.

Note: Adjust step 1 and 2 to your implementation of the preprocessing step if you don't use pipeline API from `sklearn`.

In [ ]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`